# Global setup and package installation used in most phases

## Colab + GPU Detection Utilities

In [1]:
import subprocess

def is_running_in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

def get_available_gpu_memory_gb():
    try:
        output = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.free", "--format=csv,nounits,noheader"],
            encoding="utf-8"
        )
        free_mem_mb = int(output.strip().split("\n")[0])
        return free_mem_mb / 1024
    except Exception:
        return 0.0


## install dependencies

In [ ]:
if is_running_in_colab():
    # Install the required packages
    !pip install kagglehub pandas
    !pip install -q transformers accelerate bitsandbytes sentencepiece pydantic huggingface_hub xformers
    !pip install regex json5
    !pip install dspy  sentence-transformers transformers accelerate scikit-learn -q

else:
    %pip install kagglehub pandas
    %pip install -q transformers accelerate sentencepiece pydantic huggingface_hub xformers
    #%pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121
    #%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    %pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
    %pip install -U bitsandbytes
    %pip install regex json5
    # 📦 INSTALL REQUIREMENTS
    %pip install dspy  sentence-transformers transformers accelerate scikit-learn -q



## Login to huggingface

In [2]:
from huggingface_hub import login
import os

# Set your token here securely or prompt for it in Colab
# Recommended: store in Colab secrets or environment variable
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")


if not HF_TOKEN:
    if is_running_in_colab():
        # If running in Colab, use the Colab secrets
        try:
            from google.colab import userdata
            HF_TOKEN = userdata.get('HF_TOKEN')
            if not HF_TOKEN:
                raise ValueError("⚠️ Hugging Face token not found in Colab secrets.")
            print("🔑 Hugging Face token found in Colab secrets.")
        except ImportError:
            print("⚠️ Unable to authenticate in Colab. Please set your Hugging Face token manually.")
    else:
        # Prompt for token if not set in environment
        print("🔑 Please enter your Hugging Face token:")
        # For Colab or local prompt input
        HF_TOKEN = input("🔑 Enter your Hugging Face token: ").strip()

login(token=HF_TOKEN)


## Setup Kaggle Credentials

In [3]:
import shutil

def setup_kaggle_credentials():
    kaggle_path = os.path.expanduser('~/.kaggle/kaggle.json')
    if not os.path.exists(kaggle_path):
        from google.colab import files
        print("📂 Upload kaggle.json file...")
        uploaded = files.upload()
        os.makedirs(os.path.dirname(kaggle_path), exist_ok=True)
        for filename in uploaded.keys():
            shutil.move(filename, kaggle_path)
        os.chmod(kaggle_path, 0o600)
        print(f"✅ Kaggle credentials setup at {kaggle_path}")
    else:
        print(f"✅ Kaggle credentials already exist at {kaggle_path}")

setup_kaggle_credentials()

✅ Kaggle credentials already exist at C:\Users\rubyj/.kaggle/kaggle.json


##  Load mistral-Instruct with Fallback to Quantized

### SimpleLM-Like Class for DSPy

In [4]:
from dspy import BaseLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

class SimpleLM(BaseLM):
    def __init__(self, model_name, tokenizer, model, max_new_tokens=2048):
        super().__init__(model=model_name)
        self.tokenizer = tokenizer
        self._model = model         # internal model
        self.model = model_name     # string for DSPy to parse with .split()
        self.max_new_tokens = max_new_tokens
        self.kwargs = {}

    def __call__(self, messages, **kwargs):
        # DSPy passes `messages=[{"role": "user", "content": "..."}]`
        if isinstance(messages, list):
            prompt = "\n".join([m["content"] for m in messages])
        else:
            raise ValueError("Expected messages as a list of dicts with 'content'.")

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self._model.device)
        outputs = self._model.generate(
            **inputs,
            max_new_tokens=self.max_new_tokens,
            do_sample=False,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id
        )
        output_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        return output_text


### Use It in Your DSPy Project (with Quantized Mistral)

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dspy import configure

# 🔧 Load quantized or full model manually
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
hf_token = HF_TOKEN

has_cuda = torch.cuda.is_available()
free_mem = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3) if has_cuda else 0
print(f"💻 CUDA: {has_cuda} | GPU Memory: {free_mem:.2f} GB")

device_map = {"": 0} if has_cuda else "cpu"
use_4bit = has_cuda and free_mem < 24

# Set quantization config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True if use_4bit else False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
) if use_4bit else None


tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=quant_config,
    torch_dtype=torch.float16 if not quant_config else None,
    token=hf_token,
    trust_remote_code=True
).eval()

# ✅ Plug into DSPy
lm = SimpleLM(model_name=model_name, tokenizer=tokenizer, model=model, max_new_tokens=2048)
configure(lm=lm)


💻 CUDA: True | GPU Memory: 15.92 GB


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Utilities

### Configurations  

In [6]:
# ==============================
# 🛠 CONFIGURATION
# ==============================

class Config:
    DATASET_DOWNLOAD_DIR = "datasets"
    JSON_OUTPUT_DIR = "json_outputs_run3"
    JSON_OUTPUT_NORMALIZED_DIR = "json_outputs_run3/normalized"



## Utility to save json to a folder

In [7]:
import json
import os
# 📦 Save JSON Output with Safety
def save_json_output(data, output_path: str, indent: int = 4, overwrite: bool = True):
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)

    if os.path.exists(output_path):
        if overwrite:
            os.remove(output_path)
        else:
            raise FileExistsError(f"File {output_path} already exists and overwrite=False.")

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=indent, ensure_ascii=False)

    print(f"✅ Saved output to {output_path}")


## Utility to load file

### load_ndjson_file() (for resume/jd input)

In [8]:
from pathlib import Path
from typing import List


def load_ndjson_file(file_path: Path) -> List[dict]:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file if line.strip()]


### load_json_file() (for checkpoint & metadata)

In [9]:
from pathlib import Path


def load_json_file(file_path: Path) -> dict:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)


# Phase 2 -	Parse resume/JD into JSON structured scheme

## DSPy Signature and Module

In [23]:
from dspy.utils.callback import Callback
import dspy

# ✅ Define a custom callback that works in DSPy 2.6.24
class PromptLogger(Callback):
    def on_start(self, caller, **kwargs):
        input_data = kwargs.get("input", None)
        if input_data:
            print("\n📤 Prompt Sent to LLM:\n", input_data)

    def on_end(self, caller, **kwargs):
        output_data = kwargs.get("output", None)
        if output_data:
            print("\n📥 Raw Output from LLM:\n", output_data)

# ✅ Register the callback
dspy.settings.configure(callbacks=[PromptLogger()])


ImportError: cannot import name 'Callback' from 'dspy.utils.callback' (c:\Ruby\projects\AI-resume-agent\.venv\Lib\site-packages\dspy\utils\callback.py)

In [18]:
# Updated parser function: full resume → JSON in one shot using DSPy + Mistral
from typing import Dict
from dspy import Predict, InputField, OutputField, Signature
    
class ResumeJsonSignature(Signature):
    resume_text = InputField(desc="Full resume as plain text")
    extracted_json = OutputField(desc="Parsed JSON output for the resume")
  

In [19]:
from dspy import InputField, OutputField, Signature, ChainOfThought, configure, BaseLM
# ✅ Prompt Template for Instruct Format
class ResumeParser(ChainOfThought):
    def __init__(self):
        super().__init__(ResumeJsonSignature)

    def format(self, inputs):
        return {
            "resume_text": f"""[INST] Convert the following resume into structured JSON with this format:

{{
  "summary": string,
  "skills": [string],
  "education": [{{"degree": string, "field": string, "institution": string, "year": string}}],
  "experience": [{{"job_title": string, "company": string, "start_date": string, "end_date": string, "description": string}}]
}}

Resume:
{inputs['resume_text']}

Return your output as: {{ "extracted_json": {{ ... }} }} [/INST]
"""
        }

In [20]:
def parse_resume_to_json(resume_text: str):
    parser = ResumeParser()
    result = parser(resume_text=resume_text)
    print("📤 Prompt Sent to Model:\n", result.inputs['resume_text'])
    print("\n📥 LLM Output:\n", result.extracted_json)
    
    try:
        return json.loads(result.extracted_json)
    except Exception as e:
        print("❌ Failed to parse JSON:", e)
        return {}

## Test with Sample Resume

In [21]:
sample_resume = """
John Doe

SUMMARY
Experienced software engineer with 10+ years...

PROFESSIONAL EXPERIENCE
Senior Software Engineer, ABC Corp
Built microservices in Java and deployed on Kubernetes.

EDUCATION
MS in Computer Science, XYZ University

SKILLS
Java, React, Spring Boot, SQL, Docker
"""


json_output = parse_resume_to_json(sample_resume)
print("\n✅ Parsed JSON:\n", json.dumps(json_output, indent=2))

AdapterParseError: LM response cannot be serialized to a JSON object.

Adapter JSONAdapter failed to parse the LM response. 

LM Response: Y 

Expected to find output fields in the LM response: [reasoning, extracted_json] 

